In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import cv2
import torch.nn.functional as F
import numpy as np

In [2]:
# Load the FashionMNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='data/',
                                                   train=True,
                                                   transform=transforms.ToTensor(),
                                                   download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='data/',
                                                  train=False,
                                                  transform=transforms.ToTensor(),
                                                  download=True)

# Create data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


In [3]:
# Define the AlexNet model
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [4]:

# Initialize the model, loss function and optimizer
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/10], Step [{i+1}/600], Loss: {loss.item():.4f}')
    torch.save(model, './model1_'+str(epoch)+'.pt')
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')


Epoch [1/10], Step [100/600], Loss: 2.1238
Epoch [1/10], Step [200/600], Loss: 1.0172
Epoch [1/10], Step [300/600], Loss: 0.9874
Epoch [1/10], Step [400/600], Loss: 0.5199
Epoch [1/10], Step [500/600], Loss: 0.7290
Epoch [1/10], Step [600/600], Loss: 0.4914
Epoch [2/10], Step [100/600], Loss: 0.6391
Epoch [2/10], Step [200/600], Loss: 0.5696
Epoch [2/10], Step [300/600], Loss: 0.4751
Epoch [2/10], Step [400/600], Loss: 0.4824
Epoch [2/10], Step [500/600], Loss: 0.3527
Epoch [2/10], Step [600/600], Loss: 0.4732
Epoch [3/10], Step [100/600], Loss: 0.4342
Epoch [3/10], Step [200/600], Loss: 0.3729
Epoch [3/10], Step [300/600], Loss: 0.4176
Epoch [3/10], Step [400/600], Loss: 0.4544
Epoch [3/10], Step [500/600], Loss: 0.5264
Epoch [3/10], Step [600/600], Loss: 0.2110
Epoch [4/10], Step [100/600], Loss: 0.4278
Epoch [4/10], Step [200/600], Loss: 0.3322
Epoch [4/10], Step [300/600], Loss: 0.3313
Epoch [4/10], Step [400/600], Loss: 0.4157
Epoch [4/10], Step [500/600], Loss: 0.3315
Epoch [4/10

In [4]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_model = AlexNet()
test_model = torch.load('model1_9.pt', map_location=torch.device('cpu'))
test_model = test_model.to(device)


In [5]:

# means = train_dataset.data.mean(axis = (0,1,2)) / 255
# stds = train_dataset.data.std(axis = (0,1,2)) / 255

test_transforms = transforms.Compose([
                           transforms.ToTensor(),
                        #    transforms.Normalize()
                       ])

def prepare_an_img(img):
    img = cv2.imread(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = cv2.resize(img, (32, 32))
    img = test_transforms(img).float()
    img = torch.unsqueeze(img, 0)
    return img


# The classes are: 

# 0	T-shirt/top

# 1	Trouser

# 2	Pullover

# 3	Dress

# 4	Coat

# 5	Sandal

# 6	Shirt

# 7	Sneaker

# 8	Bag

# 9	Ankle boot


coat = prepare_an_img('coat.jpg')
coat = coat.to(device)
y_pred = F.softmax(test_model(coat)[0], dim=1)
print(torch.argmax(y_pred))

sneaker = prepare_an_img('sneaker.jpg')
sneaker = sneaker.to(device)
y_pred = F.softmax(test_model(sneaker)[0], dim=1)
print(torch.argmax(y_pred))


trouser = prepare_an_img('trouser.jpg')
trouser = trouser.to(device)
y_pred = F.softmax(test_model(trouser)[0], dim=1)
print(torch.argmax(y_pred))

RuntimeError: shape '[-1, 784]' is invalid for input of size 3072